# Building a GCMC model...

This script uses MC moves for adding and removing particles in the Grand-canonical ensemble.    
    
This simulation models a lattice-gas, using the chemical potential of each species to define the acceptance criteria.    
    
For more information, see <i> Understanding molecular simulations (Frenkel & Smit), Chapter 5.6 </i>

We'll start with simply (1) adding/removing particles, then introduce (2) a 

## Imports

In [ ]:
# standard library imports
from math import ceil, exp, floor, sqrt, pi
from random import random, randint

# related third party imports
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# local application/library specific imports
from utilities import draw_lattice_configuration, Histogram

%matplotlib inline

## Simulation parameters

In [ ]:
# simulation parameters
length                   = 10                # number of sites per side
sites                    = length * length   # total number of sites
particles                = 10                # number of particles
cycles                   = 1000              # number of MC cycles
add_remove_probability   = 0.5               # probability of add/remove move
drawing_frequency        = 100               # draw configuration every

## Initialize simulation

In [ ]:
def init_simulation(length, particles):
    occupancy = np.zeros([length, length])
    coordinates = np.zeros([2, particles])

    placed_particles = 0
    for x in range(length):
        for y in range(length):
            if placed_particles < particles:
                coordinates[0, placed_particles] = x
                coordinates[1, placed_particles] = y
                occupancy[x, y] = 1
                placed_particles += 1
                
    return coordinates, occupancy

In [ ]:
coordinates, occupancy = init_simulation(length, particles)          
draw_lattice_configuration(coordinates, length)

## Create histogram for particle concentration(s)

In [ ]:
value_range        = [0, 1]    # (min, max) particle conc. [frac.]
increment          = 0.025     # bin-width
output_frequency   = 100       # cycles-per-calculation

histogram = Histogram(value_range, increment, output_frequency)

In [ ]:
def set_phi():
    # set parameters for particle concentration(s), phi
    sample_start = 100.            # step to start sampling phi
    average_phi = 0.               # holds average phi
    sample_count = 0               # holds number of times sampled
    return average_phi, sample_count, sample_start

average_phi, sample_count, sample_start = set_phi()

# (1) Adding/removing particles

## MC simulation

In [ ]:
def addition_move(length, occupancy, particles, coordinates):
    # randomly-adds a particle to the lattice
    addition = False
    if random() < 0.5:    # ADDITION MOVE
        x = randint(0, length - 1)
        y = randint(0, length - 1)
        if occupancy[x, y] == 0:
            particles += 1
            coordinates = np.c_[coordinates, [[x], [y]]]
            occupancy[x, y] = 1
            addition = True
    return occupancy, particles, coordinates, addition

def remove_move(length, occupancy, particles, coordinates):
    # randomly removes a particle from lattice
    if random() < particles / float(sites):    # probability of removal is proportional to phi
        particles -= 1
        particle = randint(0, particles - 1)
        occupancy[coordinates[0, particle], coordinates[1, particle]] = 0
        np.delete(coordinates, np.s_[particle], axis=1)
    return occupancy, particles, coordinates

def displacement_move(sites, particles, length, occupancy, coordinates):
    # randomly-displaces particles, if any within the lattice
    if particles > 0:
        for sub_cycle in range(sites):
            particle = randint(0, particles - 1)    # choose random particle
            x = randint(0, length - 1)              # choose random x-position
            y = randint(0, length - 1)              # choose random y-position
            if occupancy[x, y] == 0:                # empty old cell
                occupancy[coordinates[0, particle], coordinates[1, particle]] = 0
                coordinates[0, particle] = x        # save the new position
                coordinates[1, particle] = y
                occupancy[x, y] = 1                 # update occupancy
    return occupancy, coordinates

def run_simulation(cycles, add_remove_probability, occupancy, particles,
                   coordinates, length, sites, drawing_frequency,
                   sample_start, average_phi, sample_count, histogram
                  ):
    for cycle in range(cycles):
        if random() < add_remove_probability:
            occupancy, particles, coordinates, addition = addition_move(length, occupancy, particles, coordinates)
            if addition == False:
                occupancy, particles, coordinates = remove_move(length, occupancy, particles, coordinates)
        occupancy, coordinates = displacement_move(sites, particles, length, occupancy, coordinates)

        if cycle % drawing_frequency == 0:              # draw configuration
            print('cycle:\t%s' % cycle)
            draw_lattice_configuration(coordinates, length)

        if cycle > sample_start:                        # sample average phi
            average_phi += particles / float(sites)
            sample_count += 1
            histogram.add_data(particles / float(sites))            
    return histogram, sample_count, average_phi

In [ ]:
histogram, sample_count, average_phi = run_simulation(cycles, add_remove_probability, occupancy, particles,
                                                      coordinates, length, sites, drawing_frequency,
                                                      sample_start, average_phi, sample_count, histogram)

## Analyse results

In [ ]:
phi1 = average_phi / float(sample_count)
print('average phi:\t%s' % phi1)

h1 = histogram
simulated1 = h1.histogram / (h1.count * h1.increment)   # normalize
plt.bar(h1.values, simulated1, width=h1.increment)

# plot theoretical dist.--gaussian with mean=0.5 and variance=0.25
theoretical = [exp(-2 * sites * (i - 0.5) ** 2) * sqrt(2 * sites / pi)
               for i in h1.values
              ]
plt.plot(h1.values, theoretical, 'r')
plt.show()

# (2) Activity-based

This time, we'll use activity to measure the chemical potential:    
$$Z = \frac{e^{\beta \mu}}{\Lambda^3}$$    
which serves in the accept/reject criteria for adding/removing particles.

In [ ]:
activity = 1.

Now, we add the acceptance/rejection criteria in the model:

In [ ]:
def addition_move(length, occupancy, particles, coordinates):
    # randomly-adds a particle to the lattice
    addition = False
    if random() < 0.5:    # ADDITION MOVE
        if random() < activity * sites / (particles + 1):
            x = randint(0, length - 1)
            y = randint(0, length - 1)
            if occupancy[x, y] == 0:
                particles += 1
                coordinates = np.c_[coordinates, [[x], [y]]]
                occupancy[x, y] = 1
                addition = True
    return occupancy, particles, coordinates, addition

def remove_move(length, occupancy, particles, coordinates):
    # randomly removes a particle from lattice
    if random() < particles / float(sites * activity):    
        particles -= 1
        particle = randint(0, particles - 1)
        occupancy[coordinates[0, particle], coordinates[1, particle]] = 0
        np.delete(coordinates, np.s_[particle], axis=1)
    return occupancy, particles, coordinates

## Run simulation

In [ ]:
# initialize simulation
coordinates, occupancy = init_simulation(length, particles)    
# create histogam
histogram = Histogram(value_range, increment, output_frequency)
# set phi
average_phi, sample_count, sample_start = set_phi()
# run simulation
histogram, sample_count, average_phi = run_simulation(cycles, add_remove_probability, occupancy, particles,
                                                      coordinates, length, sites, drawing_frequency,
                                                      sample_start, average_phi, sample_count, histogram)

## Analyse result

In [ ]:
phi2 = average_phi / float(sample_count)
print('average phi:\t%s' % phi2)

h2 = histogram
simulated2 = h2.histogram / (h2.count * h2.increment)   # normalize
plt.bar(h1.values, simulated1, width=h1.increment)
plt.bar(h2.values, simulated2, width=h2.increment, color='r')
plt.plot(h1.values, theoretical, 'r')
plt.show()

# (3) Ising model

This model gives particles favorable interactions with neighbors. These interactions account for the accept/reject probabilities for add/remove and random-displacement moves.    
    
Now we'll need a function to calculate the number of nearest neighbors:

In [ ]:
def nearest_neighbors(x, y, occupancy, length):
    # find cells to left and right of a lattice site, accounting for
    # periodic boundary conditions
    if x == 0:
        left = length - 1
    else:
        left = x - 1
    if x == length - 1:
        right = 0
    else:
        right = x + 1
    neighbors = occupancy[left, y] + occupancy[right, y]
    # find cells to top and bottom of a lattice site, accounting for
    # periodic boundary conditions
    if y == 0:
        down = length - 1
    else:
        down = y - 1
    if y == length - 1:
        up = 0
    else:
        up = y + 1
    neighbors += occupancy[x, down] + occupancy[x, up]
    
    return neighbors

Our simulation becomes a bit more complicated. We're now adding the following parameters:

In [ ]:
length         = 15
sites          = length ** 2
particles      = sites          # lattice is now full
temperature    = 0.53           # temperature
mu             = -2.            # chemical potential
activity       = exp(mu / temperature)

add_rem_cycles = 100
draw_frequency = 100

First we must initialize the lattice and calculate the potential energy.

In [ ]:
coordinates, occupancy = init_simulation(length, particles)          

U = 0
for x in range(length):
    for y in range(length):
        U = U - occupancy[x, y] * nearest_neighbors(x, y, occupancy, length)
U = U / 2.    # adjust for over-counting

print('internal energy (U) :\t%s' % U)
draw_lattice_configuration(coordinates, length)

histogram = Histogram(value_range, increment, output_frequency)
average_phi, sample_count, sample_start = set_phi()

Now, to accept/reject moves we first calculate a proposed move's effect upon the interanl energy. There are now also multiple add/remove moves per cycle.

In [ ]:
def addition_move(length, occupancy, particles, coordinates, U):
    # randomly-adds a particle to the lattice
    addition = False
    if random() < 0.5:    # ADDITION MOVE
        x = randint(0, length - 1)
        y = randint(0, length - 1)
        if occupancy[x, y] == 0:
            delta_U = - nearest_neighbors(x, y, occupancy, length)
            if random() < exp(-delta_U / temperature) * activity * sites / (particles + 1):
                U += delta_U
                particles += 1
                coordinates = np.c_[coordinates, [[x], [y]]]
                occupancy[x, y] = 1
                addition = True
    return occupancy, particles, coordinates, addition, U

def remove_move(length, occupancy, particles, coordinates, U):
    # randomly removes a particle from lattice
    if random() < particles / float(sites):    # probability of removal is proportional to phi
        particle = randint(0, particles - 1)
        x = coordinates[0, particle]
        y = coordinates[1, particle]
        delta_U = nearest_neighbors(x, y, occupancy, length)
        if random() < exp(-delta_U / temperature) * particles / (sites * activity):
            U += delta_U
            occupancy[coordinates[0, particle], coordinates[1, particle]] = 0
            np.delete(coordinates, np.s_[particle], axis=1)
            particles -= 1
    return occupancy, particles, coordinates, U

def displacement_move(sites, particles, length, occupancy, coordinates, U):
    # randomly-displaces particles, if any within the lattice
    if particles > 0:
        for sub_cycle in range(sites):
            particle = randint(0, particles - 1)    # choose random particle
            x_p = coordinates[0, particle]
            y_p = coordinates[1, particle]
            x = randint(0, length - 1)              # choose random x-position
            y = randint(0, length - 1)              # choose random y-position
            if occupancy[x, y] == 0:
                # CALCULATING CHANGE IN POTENTIAL ENERGY
                # loss in energy at current particle position
                delta_U = nearest_neighbors(x_p, y_p, occupancy, length)
                # gain in energy at the new particle position
                occupancy[x_p, y_p] = 0
                delta_U += - nearest_neighbors(x, y, occupancy, length)
                occupancy[x_p, y_p] = 1
                if random() < exp(-delta_U / temperature):
                    U += delta_U
                    occupancy[x_p, y_p] = 0
                    coordinates[0, particle] = x        # save the new position
                    coordinates[1, particle] = y
                    occupancy[x, y] = 1                 # update occupancy
    return occupancy, coordinates, U

def run_simulation(cycles, add_remove_probability, occupancy, particles,
                   coordinates, length, sites, drawing_frequency,
                   sample_start, average_phi, sample_count, histogram,
                   add_rem_cycles, U
                  ):
    for cycle in range(cycles):
        for sub_cycle in range(add_rem_cycles):
            if random() < add_remove_probability:
                occupancy, particles, coordinates, addition, U = addition_move(length, occupancy, 
                                                                               particles, coordinates, U)
                if addition == False:
                    occupancy, particles, coordinates, U = remove_move(length, occupancy, particles,
                                                                       coordinates, U)
        occupancy, coordinates, U = displacement_move(sites, particles, length, occupancy,
                                                      coordinates, U)

        if cycle % drawing_frequency == 0:              # draw configuration
            print('cycle :\t%s\t\tinternal energy (U) :\t%s' % (cycle, U))
            draw_lattice_configuration(coordinates, length)

        if cycle > sample_start:                        # sample average phi
            average_phi += particles / float(sites)
            sample_count += 1
            histogram.add_data(particles / float(sites))            
    return histogram, sample_count, average_phi

## Run simulation

In [ ]:
histogram, sample_count, average_phi = run_simulation(cycles, add_remove_probability, occupancy, particles,
                                                      coordinates, length, sites, drawing_frequency,
                                                      sample_start, average_phi, sample_count, histogram,
                                                      add_rem_cycles, U)

## Analyse result

In [ ]:
phi3 = average_phi / float(sample_count)
print('average phi:\t%s' % phi3)

h3 = histogram
simulated3 = h3.histogram / (h3.count * h3.increment)   # normalize
plt.bar(h1.values, simulated1, width=h1.increment)
plt.bar(h2.values, simulated2, width=h2.increment, color='r')
plt.bar(h3.values, simulated3, width=h3.increment, color='g')
plt.plot(h1.values, theoretical, 'r')
plt.show()

# (4) Even better models...

These MC simulations have all consisted of a <i>single</i> stochastic process, evaluating the energy of the system, accepting/rejecting moves according to the temperature.     
    
Running multiple simulations at temperatures separated by a relatively small $\Delta T$, one can obtain an energy histogram from the overlap between the two simulations. This is called <b><i>parallel tempering</b></i> We can swap the two configurations to update the overall system, because Markov chains shouldn't have any memory of the past. The Metropolis-Hastings criterion determines whether the <i>swap</i> is accepted:    
$$p = min\Bigg(1, \frac{exp\big(-\frac{E_j}{k T_i} -\frac{E_i}{k T_j}\big)}{exp\big(-\frac{E_i}{k T_i} -\frac{E_j}{k T_j}\big)}\Bigg) = min\Bigg( 1, e^{E_i - E_j}\big(\frac{1}{k T_i} - \frac{1}{k T_j} \big) \Bigg)$$
Carefully selected temperatures and number of systems can result in improved mixing properties, however excessive numbers of systems can be detrimental due to <i>lateral diffusion</i>.    
    
Parallel tempering is also known as <i><b>replica exchange MCMC sampling</b></i>. These methods essentially make high temperature configurations available at lower temperatures, improving the precision of specific heat calculations, which are not alway handled well by the canonical ensemble.